In [ ]:
from Bio import Align
from Bio.Seq import Seq
import json
from six.moves.urllib.request import urlopen

pdb_id = "1MA3"
chain = "A"


with open(f'sbna_results/{pdb_id}/{chain}/{pdb_id}_monomer/FinalSum', 'r') as f:
    # convert final_sum to dataframe
    final_sum = f.readlines()
    final_sum = [i.split() for i in final_sum]
    final_sum = [(i[0][:3], i[0][3:], i[1]) for i in final_sum] 
    final_sum = pd.DataFrame(final_sum, columns=['res', 'num', 'score'])
    final_sum['num'] = final_sum['num'].astype(int)
    final_sum['score'] = final_sum['score'].astype(float)
    final_sum['res_code'] = final_sum['res'].apply(lambda x: seq1(x))
    # final sum is res (3 letter code), num (residue number), score and res_code (1 letter code) - dataframe
final_sum

,res,num,score,res_code
0,MET,1,-1.907039,M
1,GLU,2,-3.656199,E
2,ASP,3,-3.559218,D
3,GLU,4,-2.492862,E
4,ILE,5,-2.210304,I
...,...,...,...,...
244,ARG,248,-3.954506,R
245,LEU,249,-2.529610,L
246,ARG,250,-1.794944,R
247,SER,251,-3.967452,S


In [ ]:
def get_uniprot_id(pdb_id, chain):
    # read pdb file 
    with open(f'pdb_files/{pdb_id}.pdb', 'r') as f:
        pdb_text = f.read()
        for line in pdb_text.split('\n'):
            vals = line.split()
            if line.startswith('DBREF') and chain in vals[2] and 'UNP' in vals:
                return vals[6] # uniprot id is the 6th value

def fetch_sequence(uniprot_id):
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"
    response = requests.get(url)
    if response.ok:
        # Extract the sequence from the FASTA format
        lines = response.text.split('\n')
        sequence = ''.join(lines[1:])  # Skip the header line
        return sequence
    else:
        print("Failed to fetch sequence")
        return None

In [ ]:
uniprot_id = get_uniprot_id(pdb_id, chain)
sequence = fetch_sequence(uniprot_id)
sequence

'MEDEIRKAAEILAKSKHAVVFTGAGISAESGIPTFRGEDGLWRKYDPEEVASISGFKRNPRAFWEFSMEMKDKLFAEPNPAHYAIAELERMGIVKAVITQNIDMLHQRAGSRRVLELHGSMDKLDCLDCHETYDWSEFVEDFNKGEIPRCRKCGSYYVKPRVVLFGEPLPQRTLFEAIEEAKHCDAFMVVGSSLVVYPAAELPYIAKKAGAKMIIVNAEPTMADPIFDVKIIGKAGEVLPKIVEEVKRLRSEK'

In [ ]:
final_sum['sequence'] = final_sum['num'].apply(lambda x: sequence[x-1] if x-1 < len(sequence) else None)
final_sum

,res,num,score,res_code,sequence
0,MET,1,-1.907039,M,M
1,GLU,2,-3.656199,E,E
2,ASP,3,-3.559218,D,D
3,GLU,4,-2.492862,E,E
4,ILE,5,-2.210304,I,I
...,...,...,...,...,...
244,ARG,248,-3.954506,R,R
245,LEU,249,-2.529610,L,L
246,ARG,250,-1.794944,R,R
247,SER,251,-3.967452,S,S


In [ ]:
# index the sequence
tmp = [sequence[i-1] for i in final_sum["num"].to_list()]

# calculate percentage match
sum(final_sum['res_code'] == tmp)/len(tmp)


0.9718875502008032

In [ ]:
for pdb_id in os.listdir('sbna_results'):
    chains = get_dbref_data(pdb_id)
    chains = [chain['chain'] for chain in chains]
    for chain in chains:
        if not os.path.exists(f'sbna_results/{pdb_id}/{chain}/{pdb_id}_monomer/FinalSum'):
            continue
        with open(f'sbna_results/{pdb_id}/{chain}/{pdb_id}_monomer/FinalSum', 'r') as f:
            # convert final_sum to dataframe
            final_sum = f.readlines()
            final_sum = [i.split() for i in final_sum]
            final_sum = [(i[0][:3], i[0][3:], i[1]) for i in final_sum]

            final_sum = pd.DataFrame(final_sum, columns=['res', 'num', 'score'])
            final_sum['num'] = final_sum['num'].astype(int)
            final_sum['score'] = pd.to_numeric(final_sum['score'], errors='coerce') # convert score to float, can contain 'NA'
            final_sum['res_code'] = final_sum['res'].apply(lambda x: seq1(x))
            # final sum is res (3 letter code), num (residue number), score and res_code (1 letter code) - dataframe
            uniprot_id = get_uniprot_id(pdb_id, chain)
            sequence = fetch_sequence(uniprot_id)
            # index the sequence
            
            tmp = ""
            for i in final_sum["num"].to_list():
                if i <= len(sequence):
                    tmp += sequence[i-1]
            # tmp = [sequence[i-1] for i in final_sum["num"].to_list()]

            # calculate percentage match
            if sum(final_sum['res_code'] == tmp)/len(tmp) > 0.9:
                print(f"High match for {pdb_id} with chain {chain} ({sum(final_sum['res_code'] == tmp)/len(tmp)})")
            else:
                print(f"Low match for {pdb_id} with chain {chain} ({sum(final_sum['res_code'] == tmp)/len(tmp)})")

Low match for 1C26 with chain A (0.0)
Low match for 1CTQ with chain A (0.0)
Low match for 1H26 with chain A (0.0)


KeyboardInterrupt: 

In [ ]:
uniprot_id

'O30124'

In [ ]:
sequence

'MEDEIRKAAEILAKSKHAVVFTGAGISAESGIPTFRGEDGLWRKYDPEEVASISGFKRNPRAFWEFSMEMKDKLFAEPNPAHYAIAELERMGIVKAVITQNIDMLHQRAGSRRVLELHGSMDKLDCLDCHETYDWSEFVEDFNKGEIPRCRKCGSYYVKPRVVLFGEPLPQRTLFEAIEEAKHCDAFMVVGSSLVVYPAAELPYIAKKAGAKMIIVNAEPTMADPIFDVKIIGKAGEVLPKIVEEVKRLRSEK'

In [ ]:
final_sum

,res,num,score,res_code
0,SER,0,-2.312769,S
1,MET,1,-0.922705,M
2,GLU,2,-3.223279,E
3,ASN,3,-2.890906,N
4,PHE,4,-1.234061,F
...,...,...,...,...
552,GLU,428,-3.677237,E
553,THR,429,-3.388272,T
554,LEU,430,-1.833100,L
555,ASN,431,-3.314889,N


In [ ]:
pdb_id = "6RL3"
chain = "P"
uniprot_id = get_uniprot_id(pdb_id, chain)
print(uniprot_id)

P31947


In [ ]:
response = requests.get(f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta")
# read response
lines = response.text.split('\n')
lines

['>sp|P31947|1433S_HUMAN 14-3-3 protein sigma OS=Homo sapiens OX=9606 GN=SFN PE=1 SV=1',
 'MERASLIQKAKLAEQAERYEDMAAFMKGAVEKGEELSCEERNLLSVAYKNVVGGQRAAWR',
 'VLSSIEQKSNEEGSEEKGPEVREYREKVETELQGVCDTVLGLLDSHLIKEAGDAESRVFY',
 'LKMKGDYYRYLAEVATGDDKKRIIDSARSAYQEAMDISKKEMPPTNPIRLGLALNFSVFH',
 'YEIANSPEEAISLAKTTFDEAMADLHTLSEDSYKDSTLIMQLLRDNLTLWTADNAGEEGG',
 'EAPQEPQS',
 '']

In [ ]:
# get sequence mapping from pdb+chain to uniprot
response = requests.get(f"https://data.rcsb.org/rest/v1/core/polymer_entity_instance/{pdb_id}/{chain}")
response = response.json()
# response
response["rcsb_polymer_entity_instance_container_identifiers"]['auth_to_entity_poly_seq_mapping']

KeyError: 'rcsb_polymer_entity_instance_container_identifiers'

In [ ]:
get_uniprot_id("6RL3", "A")

P31947
P04637


In [ ]:


response = requests.get(f"https://rest.uniprot.org/uniprotkb/P01116.fasta")
response = response.text
# get GN value
""">sp|P01116|RASK_HUMAN GTPase KRas OS=Homo sapiens OX=9606 GN=KRAS PE=1 SV=1
MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVIDGETCLLDILDTAG
QEEYSAMRDQYMRTGEGFLCVFAINNTKSFEDIHHYREQIKRVKDSEDVPMVLVGNKCDL
PSRTVDTKQAQDLARSYGIPFIETSAKTRQRVEDAFYTLVREIRQYRLKKISKEEKTPGC
VKIKKCIIM
"""
match = re.search(r'GN=([^\s]+)', response)
if match:
    gene_name = match.group(1)
else:
    print("Gene name not found.")

Gene name not found.


In [ ]:
# maps UniProt - PDBs (one to many)
response = requests.get(f"""https://1d-coordinates.rcsb.org/graphql?query={{
  alignment(
    from:UNIPROT,
    to:PDB_INSTANCE,
    queryId:"{uniprot_id}"
  ){{
    query_sequence
    target_alignment {{
      target_id
      target_sequence
      coverage{{
        query_coverage
        query_length
        target_coverage
        target_length
      }}
      aligned_regions {{
        query_begin
        query_end
        target_begin
        target_end
      }}
    }}
  }}
}}""")
response = response.json()

for alignment in response['data']['alignment']['target_alignment']:
    # search for correct pdb and chain
    if alignment['target_id'] == f"{pdb_id}.{chain}": 
        target_sequence = alignment['target_sequence']
        aligned_regions = alignment['aligned_regions']
        break
print(aligned_regions)

[{'query_begin': 1, 'query_end': 248, 'target_begin': 6, 'target_end': 253}]


In [ ]:
uniprot_seq = fetch_sequence(uniprot_id)
if uniprot_seq:
    # print(f"Sequence for UniProt ID {uniprot_id}:")
    print(uniprot_seq[aligned_regions[0]['query_begin']-1:aligned_regions[-1]['query_end']])  # uniprot sequence
print(pdb_seq)  # pdb sequence (from sbna)
print(target_sequence)  # pdb sequence (aligned to uniprot)

SGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNFYRALMDEEDMDDVVDADEYLIPQQG
APNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLIMQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMRKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDE
GSHMASGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLIMQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMRKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNFYRALMDEEDMDDVVDADEYLIPQQG


In [ ]:
uniprot_seq

'MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLEGCPTNGPKIPSIATGMVGALLLLLVVALGIGLFMRRRHIVRKRTLRRLLQERELVEPLTPSGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNFYR

In [ ]:
aligner = Align.PairwiseAligner()
aligner.mode = 'global'
alignments = aligner.align(pdb_seq, target_sequence)

pdb_aligned, uniprot_aligned = alignments[0][0], alignments[0][1]

print("PDB: ", pdb_aligned)
print("Uni: ", uniprot_aligned)

PDB:  ----A----PNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATS-KANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLIMQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGL-K---------------VPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMRKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDE-------------------------------------
Uni:  GSHMASGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLIMQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMRKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNFYRALMDEEDMDDVVDADEYLIPQQG


In [ ]:
len(uniprot_seq)

1210

In [ ]:
pdb_residue_number = 0  # index in finalsum file
uniprot_residue_number = aligned_regions[0]['query_begin'] - aligned_regions[0]['target_begin'] + 1  # residue number in true uniprot sequence

residue_mapping = {}
for i, (pdb_residue, uniprot_residue) in enumerate(zip(pdb_aligned, uniprot_aligned)):
    if pdb_residue != '-':
        residue_mapping[pdb_residue_number] = uniprot_residue_number
        pdb_residue_number += 1
    if uniprot_residue != '-':
        uniprot_residue_number += 1
residue_mapping

{0: 690,
 1: 695,
 2: 696,
 3: 697,
 4: 698,
 5: 699,
 6: 700,
 7: 701,
 8: 702,
 9: 703,
 10: 704,
 11: 705,
 12: 706,
 13: 707,
 14: 708,
 15: 709,
 16: 710,
 17: 711,
 18: 712,
 19: 713,
 20: 714,
 21: 715,
 22: 716,
 23: 717,
 24: 718,
 25: 719,
 26: 720,
 27: 721,
 28: 722,
 29: 723,
 30: 724,
 31: 725,
 32: 726,
 33: 727,
 34: 728,
 35: 729,
 36: 730,
 37: 731,
 38: 732,
 39: 733,
 40: 734,
 41: 735,
 42: 736,
 43: 737,
 44: 738,
 45: 739,
 46: 740,
 47: 741,
 48: 742,
 49: 743,
 50: 744,
 51: 745,
 52: 746,
 53: 747,
 54: 748,
 55: 750,
 56: 751,
 57: 752,
 58: 753,
 59: 754,
 60: 755,
 61: 756,
 62: 757,
 63: 758,
 64: 759,
 65: 760,
 66: 761,
 67: 762,
 68: 763,
 69: 764,
 70: 765,
 71: 766,
 72: 767,
 73: 768,
 74: 769,
 75: 770,
 76: 771,
 77: 772,
 78: 773,
 79: 774,
 80: 775,
 81: 776,
 82: 777,
 83: 778,
 84: 779,
 85: 780,
 86: 781,
 87: 782,
 88: 783,
 89: 784,
 90: 785,
 91: 786,
 92: 787,
 93: 788,
 94: 789,
 95: 790,
 96: 791,
 97: 792,
 98: 793,
 99: 794,
 100: 795,